In [ ]:
from dask.distributed import Client, LocalCluster
import bokeh
import geopandas as gpd
import scipy
import libpysal
import momepy
import json
from dask import delayed
import glob
import dask
import dask.dataframe as dd
import momepy
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import json
from tqdm import tqdm
from shapely.geometry import Point
import networkx as nx
import pickle
import time

In [ ]:
place = "glasgow"
local_crs = 27700

In [ ]:
tessellation = gpd.read_parquet(f"./output/{place}/tessellation_p1.pq").to_crs(local_crs)
buildings = gpd.read_parquet(f"./output/{place}/buildings_p1.pq").to_crs(local_crs)
streets = gpd.read_parquet(f"./output/{place}/streets_raw.pq").explode().to_crs(local_crs).reset_index(drop=True)

In [ ]:
streets

In [ ]:
buildings_meta = buildings

In [ ]:
daskCluster = LocalCluster(threads_per_worker=1,
                n_workers=8, memory_limit='100GB')

client = Client(daskCluster)

client

In [ ]:
# # Map for comparison
# f, ax = plt.subplots(figsize=(30, 30))
# tessellation.plot(linewidth=0.8, ax=ax, edgecolor='0.8')
# streets.plot(ax=ax, color='red')
# buildings.plot(ax=ax, color='black', alpha=0.5)
# ax.set_axis_off()

# # Show the plot
# plt.show()

In [ ]:
streets['nID'] = momepy.unique_id(streets)

In [ ]:
### Building Area
buildings['building_area'] = momepy.Area(buildings).series
tessellation['tess_area'] = momepy.Area(tessellation).series

In [ ]:
continuity = momepy.COINS(streets)

# streets["continuity_stroke"] = continuity.stroke_attribute()

In [ ]:
stroke_attr = continuity.stroke_attribute()

In [ ]:
stroke_gdf = continuity.stroke_gdf()

In [ ]:
stroke_gdf.plot(stroke_gdf.length,
                figsize=(15, 15),
                cmap="viridis_r",
                linewidth=.5,
                scheme="headtailbreaks"
               ).set_axis_off()

In [ ]:
stroke_gdf.to_parquet(f"output/{place}/stroke_gdf.pq")

In [ ]:
with open(f'output/{place}/queen_1.pkl', 'rb') as file:
    queen_1 = pickle.load(file)
    
with open(f'output/{place}/queen_3.pkl', 'rb') as file:
    queen_3 = pickle.load(file)

In [ ]:
dist200 = libpysal.weights.DistanceBand.from_dataframe(buildings, 200, ids='uID')

In [ ]:
with open(f"output/{place}/dist200.pkl", "wb") as file:
    pickle.dump({'dist200': dist200}, file)

In [21]:
print('building_circular_compactness')
buildings['building_circular_compactness'] = momepy.CircularCompactness(buildings).series
print("building_elongation")
buildings['building_elongation'] =  momepy.Elongation(buildings).series
print('building_squareness')
buildings['building_squareness'] =  momepy.Squareness(buildings).series
print('building_eri')
buildings['building_eri'] = momepy.EquivalentRectangularIndex(buildings).series
print('building_orientation')
buildings['building_orientation'] = momepy.Orientation(buildings).series


building_circular_compactness
building_elongation
building_squareness


 67%|██████▋   | 367916/550840 [01:46<00:51, 3579.90it/s]/home/virgilxw/miniconda3/envs/morph_model_v1_tess/lib/python3.11/site-packages/momepy/shape.py:947: RuntimeWarning: invalid value encountered in arccos
  angle = np.degrees(np.arccos(cosine_angle))
100%|██████████| 550840/550840 [02:37<00:00, 3507.96it/s]


building_eri
building_orientation


100%|██████████| 550840/550840 [01:07<00:00, 8167.20it/s] 


In [22]:
print("building_neighbour_dist")
buildings["building_neighbour_dist"] = momepy.NeighborDistance(buildings, queen_1, 'uID').series
print("building_neighbourhood_interbuilding_distance")
buildings["building_neighbourhood_interbuilding_distance"] = momepy.MeanInterbuildingDistance(buildings, queen_1, 'uID', 3).series

building_neighbour_dist


  0%|          | 1324/550840 [00:20<2:55:40, 52.13it/s]

In [ ]:
print("tess_convexity")
tessellation["tess_convexity"] = momepy.Convexity(tessellation).series
print("tess_neighbours_200")
tessellation["tess_neighbours_200"] =  momepy.Neighbors(tessellation, dist200, 'uID', weighted=True).series
print("tess_covered_area")
tessellation["tess_covered_area"] = momepy.CoveredArea(tessellation, queen_1, "uID").series
print("tess_orientation")
tessellation["tess_orientation"] = momepy.Orientation(tessellation).series
print("tess_neighbours")
tessellation["tess_neighbours"] = momepy.Neighbors(tessellation, queen_3, 'uID', weighted=True).series

In [ ]:
print('building_circular_compactness_weight')
buildings['building_circular_compactness_weight'] = momepy.WeightedCharacter(buildings,
                                                        values='building_circular_compactness',
                                                        spatial_weights=queen_3,
                                                        unique_id='uID').series
print('building_elongation_weight')
buildings['building_elongation_weight'] = momepy.WeightedCharacter(buildings,
                                                        values='building_elongation',
                                                        spatial_weights=queen_3,
                                                        unique_id='uID').series
print('building_squareness_weight')
buildings['building_squareness_weight'] = momepy.WeightedCharacter(buildings,
                                                        values='building_squareness',
                                                        spatial_weights=queen_3,
                                                        unique_id='uID').series
print('building_eri_weight')
buildings['building_eri_weight'] = momepy.WeightedCharacter(buildings,
                                                        values='building_eri',
                                                        spatial_weights=queen_3,
                                                        unique_id='uID').series
print("building_neighbour_dist_weight")
buildings["building_neighbour_dist_weight"] = momepy.WeightedCharacter(buildings,
                                                        values='building_neighbour_dist',
                                                        spatial_weights=queen_3,
                                                        unique_id='uID').series
print("building_neighbourhood_interbuilding_distance_weight")
buildings["building_neighbourhood_interbuilding_distance_weight"] = momepy.WeightedCharacter(buildings,
                                                        values='building_neighbourhood_interbuilding_distance',
                                                        spatial_weights=queen_3,
                                                        unique_id='uID').series
print('building_orientation_weight')
buildings['building_orientation_weight'] = momepy.WeightedCharacter(buildings,
                                                        values='building_orientation',
                                                        spatial_weights=queen_3,
                                                        unique_id='uID').series
print("tess_convexity_weight")
tessellation["tess_convexity_weight"] = momepy.WeightedCharacter(tessellation,
                                                        values="tess_convexity",
                                                        spatial_weights=queen_3,
                                                        unique_id='uID').series
print("tess_covered_area_weight")
tessellation["tess_covered_area_weight"] = momepy.WeightedCharacter(tessellation,
                                                        values='tess_covered_area',
                                                        spatial_weights=queen_3,
                                                        unique_id='uID').series
print("tess_orientation_weight")
tessellation["tess_orientation_weight"] = momepy.WeightedCharacter(tessellation,
                                                        values='tess_orientation',
                                                        spatial_weights=queen_3,
                                                        unique_id='uID').series


In [ ]:
print("tess_rea_theil")
tessellation["tess_rea_theil"] = momepy.Theil(tessellation, values='tess_area', spatial_weights=queen_3, unique_id='uID').series

In [ ]:
print("building_neighbourhood_interbuilding_distance_weight")
buildings["building_neighbourhood_interbuilding_distance_weight"] = momepy.WeightedCharacter(buildings,
                                                        values='building_neighbourhood_interbuilding_distance_weight',
                                                        spatial_weights=queen_3,
                                                        unique_id='uID').series

In [ ]:
streets_w_nid = streets
streets_w_nid["nID"]= streets.index
buildings_w_nid = gpd.sjoin_nearest(buildings, streets_w_nid, how="left", max_distance=50)

In [ ]:
buildings_w_nid["Building_Street_Alignment"] = momepy.StreetAlignment(buildings_w_nid, streets_w_nid,
                                   'building_orientation', 'nID',
                                   'nID').series

In [ ]:
buildings = buildings.merge(buildings_w_nid[['uID', 'Building_Street_Alignment']], on='uID', how='left')

In [ ]:
blg_cell_align = momepy.CellAlignment(buildings, tessellation,
                                      'building_orientation', 'tess_orientation',
                                      'uID', 'uID')
buildings['building_cell_align'] = blg_cell_align.series

In [ ]:
buildings.to_parquet(f"output/{place}/buildings_tess.pq")
tessellation.to_parquet(f"output/{place}/tessellation_tess.pq")

In [ ]:
client.close()